This notebook is for a neural network (NN) on curves with positive rank and size of Sha equal to 1 and 4 and removing one BSD feature at a time. This includes both the original and log-transformed data.

In [1]:
from lib import utils, models, executor
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# fix the seed for reproducibility
seed = 42

# 1. Create balanced dataset of elliptic curves with size of the Tate-Shafarevich group equal to 4 and 9 containing all BSD features

In [2]:
# load your data here. The following ensure this will work on Windows as well as Unix
path = Path("..") / "data_files" / "sha"/ "ecq_sha_B_100_conds_1_500000_reg.parquet"
df = utils.load_data(path)
df=df[df['rank']>0]

Loaded the dataset with 120 features and 3064705 curves..


In [3]:
len_4 = df[df['sha'] == 4].shape[0]
df_balanced = df[df['sha'] == 1].sample(len_4, random_state=seed) 
df_balanced = pd.concat([df_balanced, df[df['sha'] == 4]])
df_balanced.sha.value_counts()

sha
1    18710
4    18710
Name: count, dtype: int64

In [4]:
#Get columns with all the BSD features, from which we will eventually remove one at a time
bsd_features = ['special_value', 'torsion', 'real_period', 'regulator', 'tamagawa_product', 'sha']

df_balanced_bsd = df_balanced[bsd_features].copy()

In [5]:
df_balanced_bsd.head(5)

,special_value,torsion,real_period,regulator,tamagawa_product,sha
393967,7.08730,4,0.87493,8.10043,16,1
811492,5.28675,2,0.31006,1.70505,40,1
872987,9.43579,1,1.05987,2.22569,4,1
761927,2.52192,2,0.54553,0.57786,32,1
2647019,5.41466,1,0.31870,4.24740,4,1


# 2. Delete one feature at a time on original data
The best accuracy is about 88% when removing the special value.

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
# Neural Net classifier

# choose model parameters
hidden_units = [128,64,32]

# check if we have cuda available
device = 'cuda'

# Map labels to range starting from 0
df_BSD_label_mapped = df_balanced_bsd.copy()

# Neural net training for binary classification expects labels to start at 0
df_BSD_label_mapped['sha'] = df_BSD_label_mapped['sha'].map({1: 0, 4: 1})

# choose training parameters
loss_func = nn.CrossEntropyLoss()
num_epochs = 100
lr = 0.001
evaluator = accuracy_score

# Initialize an empty DataFrame to store the results
results_df_nn = pd.DataFrame(columns=['Feature Deleted', 'Accuracy'])

for i in range(len(bsd_features)-1):
    print(f'Running model without {bsd_features[i]}..')
    df_sub = df_BSD_label_mapped.drop(columns=[bsd_features[i]])
    input_dim, output_dim = utils.get_input_output_dim(df_sub, 'sha', if_regression=False)
    model = models.VanillaNN(input_dim, hidden_units, output_dim, if_dropout=False, dropout_rate=0.3, if_batchnorm=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # reinitialize optimizer
    train_dataloader, val_dataset, test_dataset = utils.prepare_data(df_sub, 'sha', device, if_regression=False, random_state=seed)
    model, train_eval_hist, val_eval_hist, train_loss_hist, val_loss_hist = executor.train(model, train_dataloader, val_dataset, loss_func, evaluator, optimizer, num_epochs, if_regression=False, verbose=False)
    acc = executor.test(model, test_dataset, evaluator, if_regression=False)
    print(f"Test accuracy: {acc:0.3f}")
    print('----------------------------------')
    
   # Append the results to the DataFrame
    results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)

Running model without special_value..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 79 with Training CrossEntropyLoss : 0.2059, Validation CrossEntropyLoss : 0.2112. Training accuracy_score : 0.8807, Validation accuracy_score : 0.8784, to ../trained_models/model.pth.
Test accuracy: 0.885
----------------------------------
Running model without torsion..
The input dimension is 4 and the output dimension is 2.


/tmp/ipykernel_1277/1887803710.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)


Save the model from epoch 85 with Training CrossEntropyLoss : 0.4198, Validation CrossEntropyLoss : 0.4357. Training accuracy_score : 0.8020, Validation accuracy_score : 0.7982, to ../trained_models/model.pth.
Test accuracy: 0.799
----------------------------------
Running model without real_period..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 90 with Training CrossEntropyLoss : 0.4696, Validation CrossEntropyLoss : 0.4714. Training accuracy_score : 0.7798, Validation accuracy_score : 0.7805, to ../trained_models/model.pth.
Test accuracy: 0.776
----------------------------------
Running model without regulator..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 79 with Training CrossEntropyLoss : 0.3897, Validation CrossEntropyLoss : 0.3962. Training accuracy_score : 0.8287, Validation accuracy_score : 0.8259, to ../trained_models/model.pth.
Test accuracy: 0.823
----------------------------------
Running model without tamagawa_product..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 92 with Training CrossEntropyLoss : 0.4916, Validation CrossEntropyLoss : 0.5029. Training accuracy_score : 0.7684, Validation accuracy_score : 0.7642, to ../trained_models/model.pth.
Test accuracy: 0.765
----------------------------------


In [8]:
print(results_df_nn)

    Feature Deleted  Accuracy
0     special_value  0.884687
1           torsion  0.799172
2       real_period  0.776323
3         regulator  0.822688
4  tamagawa_product  0.765366


# 3. Delete one feature at a time on log-transformed data

The best accuracy is about 92% when removing the special value.

In [9]:
# Neural Net classifier on log-transformed data

# choose model parameters
hidden_units = [128,64,32]

# check if we have cuda available
device = 'cuda'

# Map labels to range starting from 0
df_BSD_label_mapped = df_balanced_bsd.copy()
for col in bsd_features[:-1]:
    df_BSD_label_mapped[col]=df_BSD_label_mapped[col].apply(np.log)

# Neural net training for binary classification expects labels to start at 0
df_BSD_label_mapped['sha'] = df_BSD_label_mapped['sha'].map({1: 0, 4: 1})

# choose training parameters
loss_func = nn.CrossEntropyLoss()
num_epochs = 100
lr = 0.001
evaluator = accuracy_score

# Initialize an empty DataFrame to store the results
results_df_nn_log = pd.DataFrame(columns=['Feature Deleted', 'Accuracy'])

for i in range(len(bsd_features)-1):
    print(f'Running model without {bsd_features[i]}..')
    df_sub = df_BSD_label_mapped.drop(columns=[bsd_features[i]])
    input_dim, output_dim = utils.get_input_output_dim(df_sub, 'sha', if_regression=False)
    model = models.VanillaNN(input_dim, hidden_units, output_dim, if_dropout=False, dropout_rate=0.3, if_batchnorm=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # reinitialize optimizer
    train_dataloader, val_dataset, test_dataset = utils.prepare_data(df_sub, 'sha', device, if_regression=False, random_state=seed)
    model, train_eval_hist, val_eval_hist, train_loss_hist, val_loss_hist = executor.train(model, train_dataloader, val_dataset, loss_func, evaluator, optimizer, num_epochs, if_regression=False, verbose=False)
    acc = executor.test(model, test_dataset, evaluator, if_regression=False)
    print(f"Test accuracy: {acc:0.3f}")
    print('----------------------------------')
    
   # Append the results to the DataFrame
    results_df_nn_log = pd.concat([results_df_nn_log, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)

Running model without special_value..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 21 with Training CrossEntropyLoss : 0.1895, Validation CrossEntropyLoss : 0.1884. Training accuracy_score : 0.9225, Validation accuracy_score : 0.9253, to ../trained_models/model.pth.
Test accuracy: 0.922
----------------------------------
Running model without torsion..
The input dimension is 4 and the output dimension is 2.


/tmp/ipykernel_1277/2569573343.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_nn_log = pd.concat([results_df_nn_log, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)


Save the model from epoch 91 with Training CrossEntropyLoss : 0.3967, Validation CrossEntropyLoss : 0.4151. Training accuracy_score : 0.8180, Validation accuracy_score : 0.8111, to ../trained_models/model.pth.
Test accuracy: 0.818
----------------------------------
Running model without real_period..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 88 with Training CrossEntropyLoss : 0.4617, Validation CrossEntropyLoss : 0.4650. Training accuracy_score : 0.7821, Validation accuracy_score : 0.7827, to ../trained_models/model.pth.
Test accuracy: 0.778
----------------------------------
Running model without regulator..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 83 with Training CrossEntropyLoss : 0.3831, Validation CrossEntropyLoss : 0.3917. Training accuracy_score : 0.8303, Validation accuracy_score : 0.8294, to ../trained_models/model.pth.
Test accuracy: 0.827
----------------------------------
Running model without tamagawa_product..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 84 with Training CrossEntropyLoss : 0.4768, Validation CrossEntropyLoss : 0.4876. Training accuracy_score : 0.7746, Validation accuracy_score : 0.7687, to ../trained_models/model.pth.
Test accuracy: 0.773
----------------------------------


In [10]:
print(results_df_nn_log)

    Feature Deleted  Accuracy
0     special_value  0.922234
1           torsion  0.818413
2       real_period  0.778193
3         regulator  0.826697
4  tamagawa_product  0.772582
